In [3]:
import sys
sys.path.insert(0, '..')
# Useful starting lines
%matplotlib inline
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [4]:
import Helpers.helpers as helper
import Helpers.cleaner as cleaner
import numpy as np
import Helpers.implementations as imp


In [5]:
x,y = helper.load_clean_data('../../data/x_train.npy', '../../data/y_train.npy')


In [6]:
print (x.shape[1])

19


In [7]:
def toDeg(x,degree):
    res = x
    for i in range(2,degree +1):
        power = np.power(x,i)
        res = np.c_[res,power]
    return res

def addConstant(x):
    return np.c_[np.ones((x.shape[0],1)),x]

In [8]:
xpow = addConstant(toDeg(x, 2))


In [9]:
x_tr,y_tr,x_te,y_te = imp.split_data(xpow,y,0.5,seed = 9)


In [10]:
max_iters = 50
gamma = .08
lambda_ = 0.1

In [11]:
print(y_tr.shape, x_tr.shape)

(111938,) (111938, 39)


In [12]:
np.zeros((x_tr.shape[1], 1)).shape

(39, 1)

In [ ]:
loss,w = imp.reg_logistic_regression(y=y_tr, tx=x_tr, lambda_=lambda_, initial_w=np.zeros((x_tr.shape[1], 1)), max_iters=max_iters, gamma=gamma)

../Helpers/implementations.py:128: RuntimeWarning: overflow encountered in exp
  exp_inv = np.exp(-1.0*t)


In [108]:
y_pred = helper.predict_labels(w,x_te)

In [109]:
y_pred.shape
print(y_pred.shape, w.shape, loss.shape, x_te.shape)

(2219, 1) (58, 1) (1, 1) (2219, 58)


In [110]:
res = np.array([(1 if(y_pred[i] == y_te[i]) else 0) for i in range(y_te.shape[0])])

In [111]:
print(res.sum()/len(res))

0.397926994142


In [172]:
for lambda_ in [0,0.0001]:
    for degree in [4,5,6]:
        xpow = addConstant(toDeg(x, degree))
        x_tr,y_tr,x_te,y_te = imp.split_data(xpow,y,0.5,seed = 9)
        loss,w = imp.ridge_regression(y_tr,x_tr,lambda_)
        y_pred = helper.predict_labels(w,x_te)
        res = np.array([(1 if(y_pred[i] == y_te[i]) else 0) for i in range(y_te.shape[0])])
        print("l= ", lambda_," d= ", degree, " score= ", res.sum()/len(res)," loss= ", loss)
        

l=  0  d=  4  score=  0.790930774797  loss=  0.076667316385
l=  0  d=  5  score=  0.792672795004  loss=  0.0757621348459
l=  0  d=  6  score=  0.789421023951  loss=  0.0768037450645
l=  0.0001  d=  4  score=  0.775315126989  loss=  0.0814069630877
l=  0.0001  d=  5  score=  0.778182760253  loss=  0.0803513763763
l=  0.0001  d=  6  score=  0.778933168958  loss=  0.0800476298695


In [176]:
#parameters
deg = 5

xpow = addConstant(toDeg(x, deg))
loss,w = imp.least_squares(y,xpow)

print(loss)

0.0759068863769


In [208]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

In [214]:
def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # get k'th subgroup in test, others in train: TODO
    # ***************************************************
    x_test = x[k_indices[k]]
    y_test = y[k_indices[k]]
    all_ind = np.array(list(range(x.shape[0])))
    rest_ind = np.setdiff1d(all_ind,k_indices[k])
    x_train = x[rest_ind]
    y_train = y[rest_ind]
    # ***************************************************
    # INSERT YOUR CODE HERE
    # form data with polynomial degree: TODO
    # ***************************************************
    x_train = addConstant(toDeg(x_train,degree))
    x_test = addConstant(toDeg(x_test,degree))
    # ***************************************************
    # INSERT YOUR CODE HERE
    # ridge regression: TODO
    # ***************************************************
    loss,w = imp.ridge_regression(y_train,x_train,lambda_)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # calculate the loss for train and test data: TODO
    # ***************************************************
    loss_tr = imp.compute_mse(y_train,x_train,w)
    loss_te = imp.compute_mse(y_test,x_test,w)
    return loss_tr, loss_te

In [210]:
# -*- coding: utf-8 -*-
"""a function of ploting figures."""
import numpy as np
import matplotlib.pyplot as plt


def cross_validation_visualization(lambds, mse_tr, mse_te):
    """visualization the curves of mse_tr and mse_te."""
    plt.semilogx(lambds, mse_tr, marker=".", color='b', label='train error')
    plt.semilogx(lambds, mse_te, marker=".", color='r', label='test error')
    plt.xlabel("lambda")
    plt.ylabel("rmse")
    plt.title("cross validation")
    plt.legend(loc=2)
    plt.grid(True)
    plt.savefig("cross_validation")

In [ ]:


def cross_validation_demo():
    seed = 1
    degree = 7
    k_fold = 4
    lambdas = np.logspace(-4, 0, 30)
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss of training data and test data
    rmse_tr = []
    rmse_te = []
    # ***************************************************
    # INSERT YOUR CODE HERE
    # cross validation: TODO
    # ***************************************************
    for lambda_ in lambdas:
        sum_te = 0
        sum_tr = 0
        for k in range(k_fold):
            mse_tr,mse_te = cross_validation(y,x,k_indices,k,lambda_, degree)
            sum_tr += np.sqrt(2*mse_tr)
            sum_te += np.sqrt(2*mse_te)
        rmse_tr.append(sum_tr/(1.0*k_fold))
        rmse_te.append(sum_te/(1.0*k_fold))
            
    cross_validation_visualization(lambdas, rmse_tr, rmse_te)

cross_validation_demo()

In [177]:
x_test,y_test = helper.load_clean_data('../../data/x_test.npy', '../../data/y_test.npy')

In [178]:
xpow_test = addConstant(toDeg(x_test, deg))

In [179]:
print(xpow.shape[1])
print(xpow_test.shape[1])

96
96


In [180]:
y_pred = helper.predict_labels(w,xpow_test)

In [187]:
y_pred = helper.changeYfromBinary(y_pred)

In [185]:
a,b, ids = helper.load_csv_data('../../data/test.csv')

In [188]:
helper.create_csv_submission(ids,y_pred,'../../data/subm2.csv')